# Classification - H4221

**Membres du groupe :**
- Adam Schlee
- Truong Son Ngo
- Jixiang Sun
- Thi Tho Vu
- Mohamed Fakroni
- Mateo Carvajal Sanchez
- Huu Thanh Tu Huynh
- Santiago Forero Gutierrez
  
## I. Introduction

La tarification de l'électricité est un aspect crucial des marchés de l'énergie, influençant à la fois les consommateurs et les fournisseurs. Dans ce projet, nous analysons les tendances des prix de l'électricité en **Colombie-Britannique (BC)** et en **Alberta (AB)**, Canada, en utilisant des mesures historiques de l'électricité. Notre objectif est de développer un modèle prédictif qui estime si le prix de l'électricité en Colombie-Britannique va **augmenter (UP)** ou **diminuer (DOWN)**.

### 1. Problématique

Étant donné un ensemble de données contenant des métriques liées à l'électricité, nous cherchons à prédire la variable **bc_price_evo**, qui indique si le prix de l'électricité en Colombie-Britannique est en hausse ou en baisse. L'ensemble de données comprend :

- **Date et heure de la mesure**
- **Prix et demande en électricité** en Colombie-Britannique et en Alberta
- **Transfert d'électricité** entre les deux régions

L'objectif final est de construire un modèle d'apprentissage automatique précis capable de classer efficacement l'évolution des prix en Colombie-Britannique.


### 2. Métrique d'évaluation

Notre modèle sera évalué en fonction de sa **précision globale**, en mettant l'accent sur la capacité à minimiser les erreurs de classification.


```math
\text{Accuracy} = \frac{\text{Number of correct predictions}}{\text{Total predictions}}
```


Une précision plus élevée indique un modèle plus performant.

### 3. Format de soumission
La soumission doit être un fichier CSV contenant les valeurs prédites de bc_price_evo pour chaque ID de test. Le format doit être le suivant:
```csv
id,bc_price_evo 
28855,UP 
28856,UP 
28857,DOWN ...
```
Ce projet explorera différentes techniques d'apprentissage automatique afin d'améliorer la précision des prédictions et de mieux comprendre les fluctuations des prix de l'électricité.

### 4. Bibliothèques
Pour ce projet, nous utiliserons certaines bibliothèques qui nous aideront à faciliter le processus.

In [ ]:
import pandas as pd                 # pandas for the data structure manipulation 
import numpy as np                  # numpy for numerical operations
import matplotlib.pyplot as plt     # matplotlib for plotting
#import sklearn as sklearn          # sklearn for machine learning and evaluation (required module will be imported later in each partie)

## II. Traitement des données

Ce document décrit une approche étape par étape pour gérer les données en utilisant Python. Le processus comprend le chargement des données, l'inspection, la gestion des valeurs manquantes et la normalisation des données.

In [ ]:
data_dir = '../data/classification/'
output_dir = '../output/classification/submission/'

### 1. Chargement des données
La première étape consiste à importer les bibliothèques nécessaires et à charger l’ensemble de données dans un DataFrame Pandas.

In [ ]:
df_train_raw = pd.read_csv(data_dir + 'train.csv', index_col=0)
df_train_raw.head()

In [ ]:
df_test_raw = pd.read_csv(data_dir + 'test.csv', index_col=0)
df_test_raw.head()

### 2. Inspection des données
En examinant les ensembles de données d'entraînement et de test, nous constatons qu'il y a 7 colonnes fournissant des informations pour les prédictions. Ces colonnes sont :
- `id` – Identifiant unique utilisé par Kaggle.
- `date` – Date à laquelle la mesure a été effectuée, entre le 15 mai 2015 et le 13 décembre 2017 (normalisée entre 0 et 1).
hour – Heure de la mesure, représentée par des périodes de 30 minutes sur 24 heures (valeurs initialement entre 0 et 47, normalisées entre 0 et 1).
- `bc_price` – Prix de l'électricité en Colombie-Britannique (normalisé entre 0 et 1).
- `bc_demand` – Demande en électricité en Colombie-Britannique (normalisée entre 0 et 1).
- `ab_price` – Prix de l'électricité en Alberta (normalisé entre 0 et 1).
- `ab_demand` – Demande en électricité en Alberta (normalisée entre 0 et 1).
- `transfer` – Transfert d'électricité programmé entre la Colombie-Britannique et l'Alberta (normalisé entre 0 et 1).
- `bc_price_evo` – Le prix de l'électricité en Colombie-Britannique augmente-t-il ou diminue-t-il par rapport aux dernières 24 heures ? Il s'agit de la variable cible (fournie uniquement dans l'ensemble d'entraînement).

Avant de procéder, il est essentiel de vérifier la structure et les propriétés des données.

In [ ]:
df_train_raw.info()
df_train_raw.describe()

In [ ]:
df_test_raw.info()
df_train_raw.describe()

Excellent, il semble que nos données soient très propres : il n’y a aucune valeur manquante et les informations complexes sont déjà normalisées. Elles pourraient donc être prêtes pour le traitement.

### 3. Préparation des données  
Pour le processus d'entraînement, nous devrons séparer les données et les prédictions (les étiquettes de la colonne `bc_price_evo`).

In [ ]:
df_train = df_train_raw.drop(columns=['bc_price_evo'])
df_train_labels = df_train_raw.loc[:, 'bc_price_evo'].copy()

df_train.head()
df_train_labels.head()

Pour l’ensemble de test, les données sont déjà prêtes, et nous pouvons utiliser les données brutes pour la prédiction. Cependant, afin de rendre le processus plus cohérent, nous allons définir une fonction de sauvegarde des soumissions, qui enregistrera nos prédictions dans un fichier CSV pour la soumission.

In [ ]:
df_test = df_test_raw.copy()

In [ ]:
def save_submission( df_test_labels, name_model  ):
    test = df_test.copy()
    test['bc_price_evo'] = df_test_labels
    test.to_csv(f'../data/classification/submission/{name_model}.csv', columns=['bc_price_evo'])

## III. Modèles de classification

### 1. Logistic Regression

La régression logistique est une méthode statistique utilisée pour des tâches de classification binaire. Contrairement à la régression linéaire, qui prédit des valeurs continues, la régression logistique prédit la probabilité qu'une entrée donnée appartienne à une classe particulière. Elle applique la **fonction sigmoïde** pour transformer les prédictions linéaires en valeurs de probabilité allant de 0 à 1.

**Comment cela fonctionne** :
1. Calcule une somme pondérée des caractéristiques d'entrée.
2. Applique la **fonction sigmoïde** pour mapper le résultat à une probabilité.
3. Utilise un seuil de décision (généralement 0,5) pour classer les points de données dans l'une des deux catégories.

La régression logistique est entièrement intégrée dans la bibliothèque Scikit Learn, dans le module `linear_model`, et nous pouvons l'utiliser comme notre modèle.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

La régression logistique de ScikitLearn prend en charge de nombreux hyperparamètres, tels que `max_iter`, `C` (inverse de la force de régularisation), `random_state`, etc. Et choisir les bons hyperparamètres pour le modèle est vraiment compliqué. Pour résoudre ce problème, nous avons décidé de procéder à une estimation de la précision : nous allons diviser l'ensemble de données d'entraînement en un sous-ensemble d'entraînement et un sous-ensemble de validation. Nous utiliserons ces sous-ensembles pour l'entraînement et la validation de notre modèle, et obtenirons la précision (bien entendu, il ne s'agit pas de la précision réelle, mais simplement d'une estimation qui nous aidera à déterminer les meilleurs hyperparamètres).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train, df_train_labels, test_size=0.20, random_state=23)
clf = LogisticRegression(max_iter=10000, random_state=0, C=15, class_weight='balanced')
clf.fit(X_train, y_train)

acc = accuracy_score(y_test, clf.predict(X_test)) * 100
print(f"Logistic Regression model accuracy: {acc:.2f}%")

Les scripts précédents montrent comment nous estimons la précision du modèle avec les hyperparamètres `max_iter=10000`, `random_state=0`, `C=15`, et `class_weight='balanced'`. En procédant de la même manière avec un autre cas, nous obtiendrons les tableaux des précisions estimées.

| Solver (d=lbfgs) | C (d=1.0) | Penalty (d=l2) | Class Weight (d=None) | Fit Intercept (d=True) | L1 Ratio (d=None) | Accuracy (%) |
|--------|----|---------|--------------|--------------|----------|--------------|
| default | default | default | default | default | default | 74.01 |
| liblinear | default | default | default | default | default | 73.94 |
| default | 0.01 | default | default | default | default | 62.97 |
| default | 0.1  | default | default | default | default | 66.77 |
| default | 10   | default | default | default | default | 75.05 |
| default | 100  | default | default | default | default | 74.96 |
| default | 15   | default | default | default | default | 75.12 |
| liblinear | 15 | l1 | default | default | default | 75.00 |
| saga | 15 | elasticnet | default | default | 0.5 | 75.08 |
| default | 15 | default | balanced | default | default | 75.13 |
| default | 15 | default | balanced | False | default | 71.65 |


En regardant le tableau ci-dessous, la précision estimée la plus élevée est obtenue avec les hyperparamètres `max_iter=10000`, `random_state=0`, `C=15`, et `class_weight='balanced'`, avec une précision de 75,12 %. Nous pouvons donc utiliser cette configuration pour notre modèle.

In [ ]:
clf = LogisticRegression(max_iter=10000, random_state=0, C=15, class_weight='balanced')
clf.fit(df_train, df_train_labels)
test_labels = clf.predict(df_test)

In [ ]:
#concat test_labels to df_test index
save_submission(test_labels, 'LogisticRegression')

Nos prédictions sont enregistrées dans un fichier CSV, et nous pouvons maintenant les soumettre sur Kaggle. Le résultat est de 0,7284, ce n'est pas très élevé, mais c'est tout de même une bonne valeur.

### 2. Decision Tree

Un arbre de décision est un algorithme d'apprentissage supervisé utilisé pour des tâches de classification et de régression. Il divise l'ensemble de données en sous-ensembles en fonction des valeurs des caractéristiques, créant une structure en arbre où chaque nœud interne représente une décision basée sur une caractéristique, les branches correspondent aux résultats des décisions, et les nœuds de feuille représentent la prédiction finale. Le modèle cherche à trouver les séparations les plus informatives pour maximiser la précision de la classification.

Les arbres de décision sont faciles à interpréter et peuvent traiter à la fois des données numériques et catégorielles. Cependant, ils sont sujets au surapprentissage, surtout lorsque la profondeur de l'arbre est grande. Des techniques telles que l'élagage, la limitation de la profondeur de l'arbre et la définition d'un nombre minimum d'échantillons pour les séparations peuvent aider à atténuer ce problème.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

L'ensemble de données a été divisé en sous-ensembles d'entraînement (70%) et de test (30%). Plusieurs configurations d'hyperparamètres ont été testées pour déterminer le modèle le plus performant. Les principaux paramètres testés incluent :

- Criterion: gini, entropy, log_loss

- Max Depth: None, 20

- Min Samples Split: 2, 5, 10, 20

- Min Samples Leaf: 1, 5

Le classificateur d'arbre de décision a été entraîné et testé sous chaque configuration, et la précision a été utilisée comme métrique d'évaluation.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train, df_train_labels, test_size=0.3, random_state=21)
dtree = DecisionTreeClassifier(random_state=1, max_depth=20)
dtree.fit(X_train, y_train)

acc = accuracy_score(y_test, dtree.predict(X_test)) * 100
print(f"Decision Tree model accuracy: {acc:.2f}%")

| Criterion (d=gini) | Max Depth (d=None) | Min Samples Split (d=2) | Min Samples Leaf (d=1) | Accuracy (%) |
|---------|---------|--------------|--------------|--------------|
| default | default | default | default | 84.63 |
| default | default | 5 | default | 84.61 |
| default | default | 10 | default | 84.27 |
| default | default | 20 | 5 | 84.09 |
| default | 20 | default | default | 85.06 |
| default | 20 | 5 | default | 84.67 |
| default | 20 | 10 | 5 | 84.38 |
| default | 20 | 20 | 5 | 84.15 |
| entropy | default | default | default | 84.73 |
| entropy | default | 5 | default | 84.60 |
| entropy | default | 10 | 5 | 84.13 |
| entropy | 20 | default | default | 84.64 |
| entropy | 20 | 5 | default | 84.45 |
| entropy | 20 | 10 | 5 | 84.05 |
| log_loss | default | default | default | 84.73 |
| log_loss | default | 5 | default | 84.60 |
| log_loss | default | 10 | 5 | 84.13 |
| log_loss | 20 | default | default | 84.64 |
| log_loss | 20 | 5 | default | 84.45 |
| log_loss | 20 | 10 | 5 | 84.05 |


La configuration la plus performante était un arbre de décision avec :

- Criterion: gini

- Max Depth: 20

- Min Samples Split: 2

- Min Samples Leaf: 1

Cette configuration a atteint une précision de 85,06 % sur l'ensemble de test. Le modèle a ensuite été réentraîné sur l'ensemble complet d'entraînement et utilisé pour générer des prédictions sur l'ensemble de test, obtenant un score de 0,8648 dans la compétition Kaggle.

In [ ]:
dtree = DecisionTreeClassifier(random_state=1, max_depth=20)
dtree.fit(df_train, df_train_labels)
dtree_test_labels = dtree.predict(df_test)

In [ ]:
save_submission(dtree_test_labels, 'DecisionTree')

Le modèle d'arbre de décision a montré de solides performances prédictives, avec la meilleure configuration obtenant une précision de 85,06 % sur l'ensemble de test et un score Kaggle de 0,8648. Les résultats indiquent que limiter la profondeur de l'arbre à 20 aide à prévenir le surapprentissage tout en maintenant une forte capacité prédictive.

L'utilisation du critère gini a produit les meilleurs résultats, bien que l'entropie et le log_loss aient obtenu des performances comparables. Le choix d'une valeur plus faible pour **min_samples_split** a permis au modèle de réaliser des séparations plus fines, mais une légère augmentation pourrait améliorer la généralisation.

Malgré la haute précision, les arbres de décision présentent certaines limitations, notamment la sensibilité aux données bruyantes et le surapprentissage lorsque la profondeur est trop grande.

### 3. Random Forest

La Forêt Aléatoire (Random Forest) est une méthode d'apprentissage en ensemble qui construit plusieurs arbres de décision et combine leurs sorties pour faire des prédictions. Cela aide à améliorer la précision et à réduire le surapprentissage par rapport à un seul arbre de décision. La classification finale est déterminée par un vote majoritaire parmi tous les arbres de décision de la forêt.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

Un classificateur Forêt Aléatoire (Random Forest) a été utilisé pour prédire bc_price_evo. L'ensemble de données a été divisé en sous-ensembles d'entraînement (80%) et de validation (20%). Le modèle a été entraîné en utilisant diverses configurations d'hyperparamètres, comprenant :

- Number of Estimators: 50, 100

- Max Depth: None, 20

- Min Samples Split: 2, 5, 10, 20

- Min Samples Leaf: 1, 5

Le modèle a été entraîné et validé sous chaque configuration, et la précision a été utilisée comme métrique d'évaluation.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_train, df_train_labels, test_size=0.2, random_state=28)

rf_model = RandomForestClassifier(random_state=1)
rf_model.fit(X_train, y_train)

acc = accuracy_score(y_test, rf_model.predict(X_test)) * 100
print(f"Decision Tree model accuracy: {acc:.2f}%")

| N Estimators (d=100) | Max Depth (d=None) | Min Samples Split (d=2) | Min Samples Leaf (d=1) | Accuracy (%) |
|---------|---------|--------------|--------------|--------------|
| 50 | default | default | default | 97.46 |
| 50 | default | default | 5 | 91.53 |
| 50 | default | 5 | default | 96.85 |
| 50 | default | 5 | 5 | 91.53 |
| 50 | default | 10 | default | 94.63 |
| 50 | default | 10 | 5 | 91.53 |
| 50 | default | 20 | default | 91.37 |
| 50 | 20 | default | default | 97.15 |
| 50 | 20 | default | 5 | 91.32 |
| 50 | 20 | 5 | default | 96.53 |
| 50 | 20 | 5 | 5 | 91.32 |
| 50 | 20 | 10 | default | 94.31 |
| 50 | 20 | 10 | 5 | 91.32 |
| 50 | 20 | 20 | default | 91.02 |
| default | default | default | default | 97.47 |
| default | default | default | 5 | 91.67 |
| default | default | 5 | default | 96.95 |
| default | default | 5 | 5 | 91.67 |
| default | default | 10 | default | 94.98 |
| default | default | 10 | 5 | 91.67 |
| default | default | 20 | default | 91.63 |
| default | 20 | default | default | 97.35 |
| default | 20 | default | 5 | 91.64 |
| default | 20 | 5 | default | 96.66 |
| default | 20 | 5 | 5 | 91.64 |
| default | 20 | 10 | default | 94.49 |
| default | 20 | 10 | 5 | 91.64 |
| default | 20 | 20 | default | 90.97 |

Les meilleurs hyperparamètres obtenus selon le tableau sont les paramètres par défaut, qui ont une précision de 97,47 %.

In [ ]:
rf_model = RandomForestClassifier(random_state=1)
rf_model.fit(df_train, df_train_labels)
rf_test_labels = rf_model.predict(df_test)

In [ ]:
save_submission(rf_test_labels, 'RandomForest')

Le modèle Random Forest a montré une performance solide avec une précision de 97,47 % sur l'ensemble de test et un score Kaggle de 0,8816. Comparé à un modèle d'arbre de décision unique, Random Forest a réduit le surapprentissage et amélioré la généralisation en exploitant l'apprentissage en ensemble.

Le modèle Random Forest a atteint une excellente précision par rapport au modèle d'arbre de décision, démontrant la puissance de l'apprentissage en ensemble. Bien que le modèle ait bien fonctionné sur l'ensemble de test, une précision de 97,47 % suggère un possible surapprentissage. Une analyse plus approfondie à l'aide de la validation croisée et de l'analyse de l'importance des caractéristiques serait bénéfique. Le nombre d'estimateurs et la profondeur des arbres ont un impact significatif sur la performance du modèle. Augmenter le nombre d'estimateurs a généralement amélioré la précision.

### 4. Support Vector Machine (SVM)

La machine à vecteurs de support (SVM) est un algorithme puissant d'apprentissage supervisé utilisé pour des tâches de classification et de régression.  
Il fonctionne en trouvant l'hyperplan optimal qui sépare au mieux les points de données en différentes classes.

Les concepts clés de SVM incluent :  
- **Maximisation de la marge** : L'objectif de SVM est de maximiser la marge entre les points de données les plus proches (vecteurs de support) et la frontière de décision.  
- **Truc du noyau** : SVM peut traiter des données non linéairement séparables en utilisant des fonctions de noyau (par exemple, linéaire, polynomial, RBF) pour projeter les données dans un espace de plus grande dimension.  
- **Régularisation (paramètre C)** : Contrôle le compromis entre l'obtention d'un faible taux d'erreur et le maintien d'une grande marge.  

SVM est entièrement intégré dans la bibliothèque Scikit Learn, dans le module `svm` avec la classe `SVC`, et nous pouvons l'utiliser comme notre modèle.

In [ ]:
from sklearn.svm import SVC
from matplotlib.pylab import RandomState

Comme le processus d'estimation de la précision que nous avons utilisé pour la régression logistique, nous allons faire la même chose pour choisir les hyperparamètres que nous souhaitons pour le modèle SVM.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train, df_train_labels, test_size=0.20, random_state=RandomState())
svm = SVC(kernel="rbf", gamma=20, C=100.0, class_weight='balanced', max_iter=100000)
svm.fit(X_train, y_train)
acc = accuracy_score(y_test, svm.predict(X_test)) * 100
print(f"Logistic Regression model accuracy: {acc:.2f}%")

Et les tableaux de précision estimée que nous obtenons sont :

| Kernel (d='rbf') | Gamma (d='scale') | C (d=1.0)     | Degree (d=3) | Coef0 (d=0.0) | Class Weight (d=None) | Max Iter (d=-1) | Accuracy (%) |
|---------|--------|--------|--------|-------|--------------|-----------|--------------|
| rbf     | 0.5    | 1.0    | default | default | default | default | 76.33 |
| rbf     | 0.5    | 10.0   | default | default | default | default | 77.39 |
| rbf     | 0.5    | 100.0  | default | default | default | default | 78.39 |
| rbf     | default| 100.0  | default | default | default | default | 79.83 |
| linear  | default| 100.0  | default | default | default | default | 74.75 |
| sigmoid | default| 100.0  | default | default | default | default | 43.20 |
| poly    | default| 100.0  | default | default | default | default | 76.42 |
| rbf     | scale  | 100.0  | default | default | default | default | 79.83 |
| rbf     | auto   | 100.0  | default | default | default | default | 76.78 |
| rbf     | 0.01   | 100.0  | default | default | default | default | 75.60 |
| rbf     | 0.1    | 100.0  | default | default | default | default | 76.76 |
| rbf     | 1      | 100.0  | default | default | default | default | 78.66 |
| rbf     | 5      | 100.0  | default | default | default | default | 80.70 |
| rbf     | 10     | 100.0  | default | default | default | default | 81.37 |
| rbf     | 20     | 100.0  | default | default | default | default | 82.05 |
| rbf     | 20     | 100.0  | default | default | balanced | 100000    | 82.12 |
| rbf     | 0.05   | 10.0   | 3      | default | default | default | 75.05 |
| rbf     | 0.05   | 10.0   | 3      | 1      | balanced | 5000      | 75.05 |


Et les meilleurs hyperparamètres obtenus selon le tableau sont : `kernel="rbf", gamma=20, C=100.0, class_weight='balanced', max_iter=-1`.

In [ ]:
svm = SVC(kernel="rbf", gamma=20, C=100.0, class_weight='balanced', max_iter=-1) #82.12%
svm.fit(df_train, df_train_labels)
test_labels = svm.predict(df_test)

In [ ]:
save_submission(test_labels, 'SVM')

Nos prédictions sont enregistrées dans un fichier CSV, et nous pouvons maintenant les soumettre sur Kaggle. Le résultat est de 0,8176, ce n'est pas très élevé, mais c'est tout de même une bonne valeur.

### 5. Naive Bayes


Naïve Bayes est un algorithme de classification probabiliste basé sur le théorème de Bayes. Il suppose que les caractéristiques sont indépendantes conditionnellement par rapport à l'étiquette de classe. Il existe différentes variantes de Naïve Bayes, et puisque nos caractéristiques sont continues, nous utilisons le **Gaussian Naïve Bayes**, qui suppose que les caractéristiques suivent une distribution normale.

Puisque nos caractéristiques sont des valeurs numériques continues, le meilleur choix est **GaussianNB** parce que :

- Il suppose que les caractéristiques sont distribuées normalement (ce qui est courant pour les données numériques).
- D'autres variantes de Naïve Bayes (comme **MultinomialNB**, **ComplementNB** et **BernoulliNB**) sont conçues pour des données discrètes/catégorielles et ne conviennent pas aux données numériques continues.

Attention : Les modèles Naïve Bayes dans scikit-learn nécessitent des étiquettes numériques au lieu d'étiquettes de chaîne (`'UP'` / `'DOWN'`). Par conséquent, nous devons transformer `'UP'` en `1` et `'DOWN'` en `0`.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
# UP → 1, DOWN → 0
label_encoder = LabelEncoder()
df_train_labels_encode = label_encoder.fit_transform(df_train_labels)

L'ensemble de données a été divisé en sous-ensembles d'entraînement (70%) et de test (30%). Étant donné que Naïve Bayes nécessite des étiquettes numériques, nous avons encodé **UP** comme 1 et **DOWN** comme 0. Le principal paramètre ajusté était **var_smoothing**, qui aide à gérer la stabilité numérique dans les calculs de probabilité.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train, df_train_labels_encode, test_size=0.3, random_state=21)

gnb = GaussianNB(var_smoothing=1e-3)
gnb.fit(X_train, y_train)

acc = accuracy_score(y_test, gnb.predict(X_test)) * 100
print(f"Gaussian Naive Bayes model accuracy: {acc:.2f}%")

| Var Smoothing (d=1e-9) | Accuracy (%) |
|---------|---------|
| default | 68.65 |
| 1e-10 | 68.65 |
| 1e-08 | 68.65 |
| 1e-07 | 68.65 |
| 1e-06 | 68.67 |
| 1e-05 | 68.71 |
| 0.0001 | 69.13 |
| 0.001 | 70.00 |
| 0.01 | 65.80 |
| 0.1 | 62.42 |

Et le meilleur hyperparamètre obtenu selon le tableau est : `var_smoothing=1e-3`.

In [ ]:
gnb = GaussianNB(var_smoothing=1e-3)
gnb.fit(df_train, df_train_labels_encode)
gnb_test_labels_encode = gnb.predict(df_test)

gnb_test_labels = label_encoder.inverse_transform(gnb_test_labels_encode)

In [ ]:
save_submission(gnb_test_labels, 'GaussianNB')

Le modèle Naïve Bayes a obtenu de moins bonnes performances que l'arbre de décision (85,06 %) et la forêt aléatoire (97,47 %). Naïve Bayes suppose que toutes les caractéristiques sont indépendantes, ce qui est rarement vrai dans les données réelles. Cette hypothèse a probablement contribué à la précision plus faible.

Le score Kaggle de 0,6945 indique que Naïve Bayes n'est pas le meilleur modèle pour cette tâche, mais qu'il sert de classificateur de base.

Les prédictions du modèle Naïve Bayes sont enregistrées dans un fichier CSV, et le résultat sur Kaggle est de 0,6945, ce qui est assez faible. Cependant, il est normal que Naïve Bayes (NB) obtienne de moins bonnes performances que d'autres modèles, tels que Random Forest, kNN ou SVM, pour plusieurs raisons, notamment :

- **Naïve Bayes suppose que les caractéristiques sont indépendantes**, ce qui est rarement vrai dans les données réelles.
- **NB calcule les probabilités indépendamment pour chaque caractéristique**, ce qui peut être inexact si les caractéristiques sont corrélées.

Ces limitations expliquent en partie pourquoi Naïve Bayes peut avoir des performances inférieures à celles d'autres modèles plus complexes.

### 6. K-Nearest Neighbors (KNN)


Le **k-Nearest Neighbors** (kNN) est un algorithme d'apprentissage non paramétrique basé sur les instances, utilisé pour des tâches de classification. Il classe un point de données donné en observant la classe majoritaire de ses **k voisins les plus proches**. L'algorithme est sensible aux métriques de distance et aux schémas de pondération, ce qui rend l'ajustement des hyperparamètres crucial pour optimiser la performance.

### Points clés du kNN :
- **Non-paramétrique** : Cela signifie qu'il n'y a pas de modèle spécifique appris pendant l'entraînement. L'algorithme conserve simplement les données d'entraînement et les utilise directement pour effectuer des prédictions.
- **Sensibilité aux métriques de distance** : Le choix de la distance (par exemple, Euclidienne, Manhattan, etc.) affecte directement la performance du modèle.
- **Paramètre k** : Le nombre de voisins à considérer est un hyperparamètre crucial pour la performance du modèle. Un petit k peut rendre le modèle sensible au bruit, tandis qu'un grand k peut rendre le modèle trop général et moins sensible aux détails.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
# UP → 1, DOWN → 0
label_encoder = LabelEncoder()
df_train_labels_encode = label_encoder.fit_transform(df_train_labels)

Un classificateur kNN a été utilisé pour prédire bc_price_evo. L'ensemble de données a été divisé en sous-ensembles d'entraînement (80%) et de validation (20%). Le modèle a été entraîné en utilisant différentes configurations d'hyperparamètres, comprenant :

- Number of Neighbors: 1, 3, 5, 7, 9, 11

- Distance Metrics: minkowski, euclidean, manhattan, chebyshev

- Weighting Methods: uniform, distance

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_train, df_train_labels_encode, test_size=0.2, random_state=28)

knn = KNeighborsClassifier(n_neighbors=9, weights="distance", metric="manhattan")
knn.fit(X_train, y_train)

acc = accuracy_score(y_test, knn.predict(X_test)) * 100
print(f"kNN model accuracy: {acc:.2f}%")

| N Neighbors (d=5) | Weights (d='uniform') | Metric (d='minkowski') | Accuracy (%) |
|---------|----------|-----------|--------------|
| 1 | uniform | minkowski | 94.64 |
| 1 | uniform | euclidean | 94.64 |
| 1 | uniform | manhattan | 94.85 |
| 1 | uniform | chebyshev | 94.50 |
| 1 | distance | minkowski | 94.64 |
| 1 | distance | euclidean | 94.64 |
| 1 | distance | manhattan | 94.85 |
| 1 | distance | chebyshev | 94.50 |
| 3 | distance | minkowski | 95.23 |
| 3 | distance | euclidean | 95.23 |
| 3 | distance | manhattan | 95.43 |
| 3 | distance | chebyshev | 95.07 |
| 5 | distance | minkowski | 95.39 |
| 5 | distance | euclidean | 95.39 |
| 5 | distance | manhattan | 95.59 |
| 5 | distance | chebyshev | 95.15 |
| 7 | distance | minkowski | 95.59 |
| 7 | distance | euclidean | 95.59 |
| 7 | distance | manhattan | 95.68 |
| 7 | distance | chebyshev | 95.29 |
| 9 | distance | minkowski | 95.52 |
| 9 | distance | euclidean | 95.52 |
| 9 | distance | manhattan | 95.95 |
| 9 | distance | chebyshev | 95.58 |
| 11 | distance | minkowski | 95.53 |
| 11 | distance | euclidean | 95.53 |
| 11 | distance | manhattan | 95.84 |
| 11 | distance | chebyshev | 95.75 |

La configuration la plus performante était un modèle kNN avec :

- **Nombre de voisins** : 9  
- **Méthode de pondération** : distance  
- **Métrique de distance** : manhattan  

Cette configuration a atteint une précision de 95,95 % sur l'ensemble de test. Le modèle a ensuite été réentraîné sur l'ensemble complet d'entraînement et utilisé pour générer des prédictions sur l'ensemble de test, obtenant un score de 0,7870 dans la compétition Kaggle.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=9, weights="distance", metric="manhattan")
knn.fit(df_train, df_train_labels_encode)
knn_test_labels_encode = knn.predict(df_test)

knn_test_labels = label_encoder.inverse_transform(knn_test_labels_encode)

In [ ]:
save_submission(knn_test_labels, 'knn_model')

Le modèle kNN a obtenu une précision solide de 95,95 % sur l'ensemble de test, ce qui indique que le modèle a bien performé sur cette tâche de classification. 

Malgré la haute précision sur l'ensemble de test, le score de 0,7870 dans la compétition Kaggle suggère que le modèle peut rencontrer des difficultés de généralisation par rapport à d'autres modèles comme **Random Forest**. Enfin, kNN est un algorithme coûteux en termes de calcul, surtout pour de grands ensembles de données, car les prédictions nécessitent des calculs de distance pour chaque instance de test.

## IV. Conclusion

Dans ce projet, plusieurs modèles d'apprentissage automatique ont été évalués pour prédire l'évolution des prix de l'électricité en Colombie-Britannique (BC). Les modèles variaient en termes de complexité, d'hypothèses et de performances. Voici un résumé de leurs résultats :

| Model | Accuracy (%) | Kaggle Score |
| --- | --- | --- |
| Logistic Regression | 75.12 | 0.7284 |
| Support Vector Machine (SVM) | 82.12 | 0.8176 |
| Decision Tree | 85.06 | 0.8648 |
| Random Forest | **97.47** | **0.8816** |
| Naïve Bayes | 70.00 | 0.6945 |
| k-Nearest Neighbors (kNN) | 95.95 | 0.7870 |


### Points clés à retenir :
- **Le classificateur Random Forest** a obtenu la meilleure précision (97,47 %) et le meilleur score dans la compétition Kaggle (0,8816). Son approche en ensemble a fourni une forte capacité de généralisation et a surpassé les autres modèles.
- **kNN** a bien performé sur l'ensemble de test (95,95 %), mais a obtenu un score Kaggle plus faible (0,7870), indiquant des problèmes potentiels de généralisation. Le modèle d'arbre de décision a également montré une précision solide (85,06 %), mais a été surpassé par Random Forest.
- **SVM** a démontré un bon équilibre entre précision (82,12 %) et généralisation (score Kaggle de 0,8176), en faisant un choix fiable pour des données structurées.
- **Naïve Bayes** a obtenu la plus faible précision (70,00 %) et le score Kaggle le plus bas (0,6945). Cela était attendu en raison de son hypothèse d'indépendance des caractéristiques, qui est rarement vraie dans les ensembles de données réels.
- **La régression logistique et Naïve Bayes** ont été les modèles les plus rapides à entraîner, mais ont manqué de puissance prédictive. **Random Forest** et **kNN**, bien que très précis, étaient plus coûteux en termes de calculs.